<a href="https://colab.research.google.com/github/mpazaryna/ai-experiments/blob/main/notebooks/Copy_of_chroma_db_doc_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain openai tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.6/508.6 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.352
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [ ]:
!pip show openai

Name: openai
Version: 1.6.1
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: llmx


In [ ]:
!unzip -q data.zip -d new_articles

In [ ]:
from google.colab import userdata
v = userdata.get('openai')

In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

In [ ]:
# load and process the text files
loader = DirectoryLoader('./new_articles/data/consolidated', glob="./*.md", loader_cls=TextLoader)
documents = loader.load()


In [ ]:
# split text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)

302

In [ ]:
texts[3]

Document(page_content='By streamlining healthcare processes, pCare helps hospitals operate more efficiently. When nurses feel like their time is valued and they are engaged in fulfilling work, they are less motivated to leave their roles. With end-to-end integration, tasks are routed directly to support staff allowing nurses to practice at the top of their license, creating a productive healthcare facility.\nJob satisfaction comes when nurses know they are having an impact and their core matters. By fostering collaboration and communication, the pCare IPS helps improve patient self-efficacy in their ability to manage care post-discharge. Nurses report “the (IPS education) view rates… makes the team much more confident in the quality of care they are able to provide.”', metadata={'source': 'new_articles/data/consolidated/pcare-nurse-retention.md'})

In [ ]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
# persist the db to disk
vectordb.persist
vectordb = None

In [ ]:
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("what is pcare?")

In [ ]:
len(docs)

4

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "What is pcare?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 pCare is a company that provides interactive patient experience solutions for healthcare organizations to engage, educate, and entertain patients. They have won awards for their interactive patient systems and have been acquired by Uniguest.


Sources:
new_articles/data/consolidated/pcare-staff-empowerment.md
new_articles/data/consolidated/pcare-pcare-achieves-hitrust-implemented-1-year-i1-certification-to-manage-data-protection-and-mitigate-cybersecurity-threats.md
new_articles/data/consolidated/pcare-about-us.md
new_articles/data/consolidated/pcare-pcare.com.md


In [ ]:
# break it down
query = "What is the pcare?"
llm_response = qa_chain(query)
# process_llm_response(llm_response)
llm_response

{'query': 'What is the pcare?',
 'result': ' pCare is a company that provides patient engagement solutions, including actionable analytics, IPS and rounding tools, and a team to collaborate with clients on setting success metrics and designing programs for optimal system utilization. They are based in Lake Success, NY and also use cookies on their website.',
 'source_documents': [Document(page_content="pCare's actionable analytics are easy to personalize through a configurable dashboard that seamlessly integrates with your EHR system. Easily set your priorities to support your own workflow and reporting needs.\nThe IPS and rounding tools from pCare are proven to help improve a healthcare organization’s performance, efficiency and patient satisfaction scores.\npCare's team is responsible for collaborating with client staff on setting the metrics for success and designing a program to ensure optimal system utilization in support of defined patient engagement/experience, care, and staff e